<a href="https://colab.research.google.com/github/Prianka-Mukhopadhyay/budget-decision-engine/blob/main/budget_decision_engine_v1_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import timedelta
np.random.seed(42)


This generator simulates:

Google + Meta ads,
diminishing returns,
creative fatigue,
lagged revenue.

In [ ]:
def generate_synthetic_marketing_data(
    start_date="2023-01-01",
    weeks=26,
    channels=("Google", "Meta"),
):
    rng = np.random.default_rng(123)  # 🔒 deterministic randomness

    dates = pd.date_range(start=start_date, periods=weeks, freq="W")

    data = []
    revenue_data = []

    base_roas = {"Google": 4.0, "Meta": 2.8}
    fatigue_start_week = 12

    for date_idx, date in enumerate(dates):
        weekly_revenue = 0

        for channel in channels:
            base_spend = 8000 if channel == "Google" else 12000
            spend = max(2000, base_spend + rng.normal(0, 800))

            roas = base_roas[channel] - 0.00008 * spend
            if channel == "Meta" and date_idx > fatigue_start_week:
                roas *= 0.85

            conversions = max(10, int((spend * roas) / 80))
            revenue = conversions * 80

            weekly_revenue += revenue

            data.append({
                "date": date,
                "channel": channel,
                "spend": round(spend, 2),
                "conversions": conversions,
                "revenue": round(revenue, 2)
            })

        revenue_data.append({
            "date": date,
            "total_revenue": round(weekly_revenue, 2)
        })

    return pd.DataFrame(data), pd.DataFrame(revenue_data)


###Generate Data

In [ ]:
ads_df, revenue_df = generate_synthetic_marketing_data()

ads_df["date"] = pd.to_datetime(ads_df["date"])
revenue_df["date"] = pd.to_datetime(revenue_df["date"])


###Weekly Aggregation

In [ ]:
weekly = (
    ads_df
    .groupby(["date", "channel"], as_index=False)
    .agg({
        "spend": "sum",
        "conversions": "sum",
        "revenue": "sum"
    })
)

weekly["roas"] = weekly["revenue"] / weekly["spend"]
weekly = weekly.sort_values(["channel", "date"])


###FEATURE ENGINEERING

In [ ]:
weekly["roas_4w"] = weekly.groupby("channel")["roas"].transform(
    lambda x: x.rolling(4, min_periods=2).mean()
)

weekly["roas_12w"] = weekly.groupby("channel")["roas"].transform(
    lambda x: x.rolling(12, min_periods=4).mean()
)

weekly["roas_decay"] = (weekly["roas_4w"] - weekly["roas_12w"]) / weekly["roas_12w"]

weekly["total_spend_week"] = weekly.groupby("date")["spend"].transform("sum")
weekly["spend_share"] = weekly["spend"] / weekly["total_spend_week"]

weekly["delta_spend"] = weekly.groupby("channel")["spend"].diff()
weekly["delta_revenue"] = weekly.groupby("channel")["revenue"].diff()
weekly["marginal_roas"] = weekly["delta_revenue"] / weekly["delta_spend"]

weekly.loc[
    (weekly["delta_spend"] <= 0) | (weekly["marginal_roas"] > 10),
    "marginal_roas"
] = np.nan


In [ ]:
latest = (
    weekly
    .sort_values("date")
    .groupby("channel")
    .tail(1)
    .reset_index(drop=True)
)


###DECISION ENGINE

In [ ]:
# THRESHOLDS = {
#     "roas_good": 3.0,
#     "roas_bad": 1.5,
#     "decay_bad": -0.15,
#     "spend_share_high": 0.6,
#     "marginal_good": 2.5,
# }
THRESHOLDS = {
    "decay_bad": -0.15,
    "marginal_good": 2.5
}


In [ ]:
def make_channel_decision(row, all_rows):
    decisions = []

    avg_roas = all_rows["roas"].mean()

    if row["roas"] < 0.7 * avg_roas:
        decisions.append("underperforming")

    if row["roas_decay"] < THRESHOLDS["decay_bad"]:
        decisions.append("fatigue")

    if pd.notnull(row["marginal_roas"]) and row["marginal_roas"] > THRESHOLDS["marginal_good"]:
        decisions.append("scale")

    return decisions


In [ ]:
latest["decision_flags"] = latest.apply(
    lambda r: make_channel_decision(r, latest),
    axis=1
)


In [ ]:
def compute_confidence(row):
    score = 0

    if abs(row["roas_decay"]) > 0.15:
        score += 30

    if pd.notnull(row["marginal_roas"]) and row["marginal_roas"] < 1.2:
        score += 30

    if "underperforming" in row["decision_flags"]:
        score += 20

    if row["spend_share"] > 0.5:
        score += 20

    if score == 0:
        score = 30

    return min(score, 100)


In [ ]:
latest["confidence_score"] = latest.apply(compute_confidence, axis=1)


###Recommendation Logic

In [ ]:
def budget_recommendation(row):
    if row["confidence_score"] >= 80:
        if "scale" in row["decision_flags"]:
            return f"🟢 Increase spend by ~${int(0.1 * row['spend']):,} per week"
        return "🔴 Reduce spend by ~15%"

    if 50 <= row["confidence_score"] < 80 and "underperforming" in row["decision_flags"]:
        return "🟠 Monitor closely; consider 5–10% reduction"

    return "🟡 Maintain current spend"


In [ ]:
latest["recommendation"] = latest.apply(budget_recommendation, axis=1)


###Explanation Logic

In [ ]:
def generate_explanation(row, all_rows):
    avg_roas = all_rows["roas"].mean()
    reasons = []

    if "underperforming" in row["decision_flags"]:
        reasons.append(
            f"ROAS ({row['roas']:.2f}) is below the channel average ({avg_roas:.2f})."
        )

    if pd.notnull(row["marginal_roas"]) and row["marginal_roas"] < 1.2:
        reasons.append(
            f"Marginal ROAS ({row['marginal_roas']:.2f}) indicates inefficient scaling."
        )

    if abs(row["roas_decay"]) < 0.05:
        reasons.append(
            "Recent performance is stable, so aggressive changes are not required."
        )

    if not reasons:
        reasons.append(
            "Performance is stable relative to historical trends."
        )

    return " ".join(reasons)


In [ ]:
latest["explanation"] = latest.apply(
    lambda r: generate_explanation(r, latest),
    axis=1
)


In [ ]:
latest[[
    "channel",
    "recommendation",
    "confidence_score",
    "explanation"
]]


,channel,recommendation,confidence_score,explanation
0,Google,🟡 Maintain current spend,30,"Recent performance is stable, so aggressive ch..."
1,Meta,🟠 Monitor closely; consider 5–10% reduction,70,ROAS (1.56) is below the channel average (2.45...


##ML

In [ ]:
import statsmodels.api as sm
from patsy import dmatrix


In [ ]:
model_data = (
    weekly
    .groupby(["channel", "date"], as_index=False)
    .agg({
        "spend": "sum",
        "revenue": "sum"
    })
)


In [ ]:
def fit_spend_response(channel_df):
    # Build spline basis for spend
    spline = dmatrix(
        "bs(spend, df=4, degree=3, include_intercept=False)",
        data=channel_df,
        return_type="dataframe"
    )

    model = sm.OLS(channel_df["revenue"], spline).fit()
    return model, spline


In [ ]:
channel_models = {}

for channel in model_data["channel"].unique():
    df_c = model_data[model_data["channel"] == channel]
    model, spline = fit_spend_response(df_c)
    channel_models[channel] = {
        "model": model,
        "data": df_c
    }


In [ ]:
for channel, obj in channel_models.items():
    print(f"\nChannel: {channel}")
    print(obj["model"].summary())



Channel: Google
                            OLS Regression Results                            
Dep. Variable:                revenue   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 7.496e+04
Date:                Thu, 08 Jan 2026   Prob (F-statistic):           2.73e-43
Time:                        21:00:40   Log-Likelihood:                -111.66
No. Observations:                  26   AIC:                             233.3
Df Residuals:                      21   BIC:                             239.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [ ]:
# def simulate_spend_change(channel, pct_change=0.1, n_points=50):
#     obj = channel_models[channel]
#     model = obj["model"]
#     df = obj["data"]

#     current_spend = df["spend"].iloc[-1]
#     spend_range = np.linspace(
#         current_spend * (1 - pct_change),
#         current_spend * (1 + pct_change),
#         n_points
#     )

#     spline = dmatrix(
#         "bs(spend, df=4, degree=3, include_intercept=False)",
#         {"spend": spend_range},
#         return_type="dataframe"
#     )

#     pred = model.get_prediction(spline)
#     summary = pred.summary_frame(alpha=0.2)  # 80% CI

#     return pd.DataFrame({
#         "spend": spend_range,
#         "revenue_mean": summary["mean"],
#         "revenue_low": summary["mean_ci_lower"],
#         "revenue_high": summary["mean_ci_upper"]
#     })

def simulate_spend_change(model_dict, channel, pct_change=0.1, n_points=50):
    obj = model_dict[channel]
    model = obj["model"]
    df = obj["data"]

    current_spend = df["spend"].iloc[-1]

    spend_range = np.linspace(
        current_spend * (1 - pct_change),
        current_spend * (1 + pct_change),
        n_points
    )

    spline = dmatrix(
        "bs(spend, df=4, degree=3, include_intercept=False)",
        {"spend": spend_range},
        return_type="dataframe"
    )

    pred = model.get_prediction(spline)
    summary = pred.summary_frame(alpha=0.2)  # 80% CI

    return pd.DataFrame({
        "spend": spend_range,
        "revenue_mean": summary["mean"],
        "revenue_low": summary["mean_ci_lower"],
        "revenue_high": summary["mean_ci_upper"]
    })


In [ ]:
# simulations = {}

# for channel in channel_models.keys():
#     simulations[channel] = simulate_spend_change(channel)

# simulations["Google"].head(), simulations["Meta"].head()
simulations = {}

for channel in channel_models.keys():
    simulations[channel] = simulate_spend_change(
        channel_models,
        channel
    )


In [ ]:
for channel, sim in simulations.items():
    print(f"\n{channel}")
    print("Expected revenue range:",
          int(sim["revenue_low"].min()),
          "→",
          int(sim["revenue_high"].max()))



Google
Expected revenue range: 21733 → 31548

Meta
Expected revenue range: 15256 → 23028


In [ ]:
# def ml_based_decision(channel, sim_df):
#     base_rev = sim_df["revenue_mean"].iloc[len(sim_df)//2]

#     # +10% spend point
#     upper = sim_df.iloc[-1]

#     expected_lift = upper["revenue_mean"] - base_rev
#     uncertainty = upper["revenue_high"] - upper["revenue_low"]

#     # Risk-adjusted signal
#     risk_ratio = uncertainty / max(expected_lift, 1)

#     if expected_lift > 1000 and risk_ratio < 1.0:
#         return {
#             "decision": "🟢 Increase spend",
#             "confidence": 80,
#             "reason": "Revenue response is strong and predictable under increased spend."
#         }

#     if expected_lift > 500 and risk_ratio < 2.0:
#         return {
#             "decision": "🟠 Monitor closely",
#             "confidence": 60,
#             "reason": "Potential upside exists but uncertainty is elevated."
#         }

#     return {
#         "decision": "🔴 Avoid scaling / consider reduction",
#         "confidence": 40,
#         "reason": "Revenue response is weak or highly uncertain."
#     }


In [ ]:
def ml_based_decision(channel, sim_df):
    base = sim_df.iloc[len(sim_df)//2]
    upper = sim_df.iloc[-1]

    base_rev = base["revenue_mean"]
    lift = upper["revenue_mean"] - base_rev

    pct_lift = lift / max(base_rev, 1)
    uncertainty = (upper["revenue_high"] - upper["revenue_low"]) / max(base_rev, 1)

    # Risk-adjusted decision using relative terms
    if pct_lift > 0.08 and uncertainty < 0.1:
        return {
            "decision": "🟢 Increase spend",
            "confidence": 80,
            "reason": "Revenue is expected to increase meaningfully with controlled uncertainty."
        }

    if pct_lift > 0.03 and uncertainty < 0.25:
        return {
            "decision": "🟠 Monitor closely",
            "confidence": 60,
            "reason": "Some upside exists, but uncertainty suggests cautious optimization."
        }

    return {
        "decision": "🔴 Avoid scaling / consider reduction",
        "confidence": 40,
        "reason": "Expected revenue lift is small relative to uncertainty."
    }


In [ ]:
final_decisions = []

for channel, sim in simulations.items():
    result = ml_based_decision(channel, sim)
    final_decisions.append({
        "channel": channel,
        "recommendation": result["decision"],
        "confidence_score": result["confidence"],
        "explanation": result["reason"]
    })

final_output = pd.DataFrame(final_decisions)
final_output


,channel,recommendation,confidence_score,explanation
0,Google,🟢 Increase spend,80,Revenue is expected to increase meaningfully w...
1,Meta,🔴 Avoid scaling / consider reduction,40,Expected revenue lift is small relative to unc...


###DATASET

In [ ]:
real_df = pd.read_csv("/content/Advertising.csv")
real_df.head()


,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [ ]:
real_long = real_df.melt(
    id_vars=["sales"],
    value_vars=["TV", "radio", "newspaper"],
    var_name="channel",
    value_name="spend"
)

real_long.rename(columns={"sales": "revenue"}, inplace=True)

# Fake a time index (required by pipeline)
real_long["date"] = pd.date_range(
    start="2022-01-01",
    periods=len(real_long),
    freq="W"
)

real_long.head()


,revenue,channel,spend,date
0,22.1,TV,230.1,2022-01-02
1,10.4,TV,44.5,2022-01-09
2,9.3,TV,17.2,2022-01-16
3,18.5,TV,151.5,2022-01-23
4,12.9,TV,180.8,2022-01-30


In [ ]:
real_models = {}

for channel in real_long["channel"].unique():
    df_c = real_long[real_long["channel"] == channel]

    spline = dmatrix(
        "bs(spend, df=4, degree=3, include_intercept=False)",
        data=df_c,
        return_type="dataframe"
    )

    model = sm.OLS(df_c["revenue"], spline).fit()

    real_models[channel] = {
        "model": model,
        "data": df_c
    }


In [ ]:
for channel, obj in real_models.items():
    print("\n======================")
    print("Channel:", channel)
    print(obj["model"].summary())



Channel: TV
                            OLS Regression Results                            
Dep. Variable:                revenue   R-squared:                       0.623
Model:                            OLS   Adj. R-squared:                  0.615
Method:                 Least Squares   F-statistic:                     80.51
Date:                Thu, 08 Jan 2026   Prob (F-statistic):           3.16e-40
Time:                        21:16:17   Log-Likelihood:                -516.17
No. Observations:                 200   AIC:                             1042.
Df Residuals:                     195   BIC:                             1059.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

In [ ]:
real_simulations = {}

for channel in real_models.keys():
    real_simulations[channel] = simulate_spend_change(
        real_models,
        channel
    )


In [ ]:
real_decisions = []

for channel, sim in real_simulations.items():
    result = ml_based_decision(channel, sim)
    real_decisions.append({
        "channel": channel,
        "recommendation": result["decision"],
        "confidence_score": result["confidence"],
        "explanation": result["reason"]
    })

pd.DataFrame(real_decisions)


""


In [ ]:
real_final_decisions = []

for channel, sim in real_simulations.items():
    result = ml_based_decision(channel, sim)
    real_final_decisions.append({
        "channel": channel,
        "recommendation": result["decision"],
        "confidence_score": result["confidence"],
        "explanation": result["reason"]
    })

real_final_output = pd.DataFrame(real_final_decisions)
real_final_output


,channel,recommendation,confidence_score,explanation
0,TV,🟠 Monitor closely,60,"Some upside exists, but uncertainty suggests c..."
1,radio,🔴 Avoid scaling / consider reduction,40,Expected revenue lift is small relative to unc...
2,newspaper,🔴 Avoid scaling / consider reduction,40,Expected revenue lift is small relative to unc...


In [ ]:
# --- Prepare synthetic data ---
synthetic_pool = weekly[["date", "channel", "spend", "revenue"]].copy()
synthetic_pool["dataset"] = "synthetic"

# --- Prepare real data (Advertising.csv) ---
real_pool = real_long[["date", "channel", "spend", "revenue"]].copy()
real_pool["dataset"] = "advertising"

# --- Combine ---
pooled_df = pd.concat([synthetic_pool, real_pool], ignore_index=True)

pooled_df.head()


,date,channel,spend,revenue,dataset
0,2023-01-01,Google,7208.70,24640.0,synthetic
1,2023-01-08,Google,9030.34,29520.0,synthetic
2,2023-01-15,Google,8736.18,28800.0,synthetic
3,2023-01-22,Google,7490.83,25440.0,synthetic
4,2023-01-29,Google,7746.72,26160.0,synthetic


In [ ]:
pooled_df.groupby(["dataset", "channel"]).size()


dataset      channel  
advertising  TV           200
             newspaper    200
             radio        200
synthetic    Google        26
             Meta          26
dtype: int64

In [ ]:
# --- Force numeric types ---
pooled_df["revenue"] = pd.to_numeric(pooled_df["revenue"], errors="coerce")
pooled_df["spend"] = pd.to_numeric(pooled_df["spend"], errors="coerce")

# Drop rows with bad revenue/spend
pooled_df_clean = pooled_df.dropna(subset=["revenue", "spend"]).reset_index(drop=True)

# --- Build spline basis ---
pooled_spline = dmatrix(
    "bs(spend, df=5, degree=3, include_intercept=False)",
    data=pooled_df_clean,
    return_type="dataframe"
)

# --- Dataset dummy (hierarchical offset) ---
dataset_dummies = pd.get_dummies(
    pooled_df_clean["dataset"],
    drop_first=True
).astype(float)

# --- Combine design matrix ---
X_pooled = pd.concat(
    [pooled_spline.reset_index(drop=True),
     dataset_dummies.reset_index(drop=True)],
    axis=1
)

# --- Convert to numpy & remove non-finite rows ---
X = X_pooled.to_numpy(dtype=float)
y = pooled_df_clean["revenue"].to_numpy(dtype=float)

mask = np.isfinite(X).all(axis=1) & np.isfinite(y)

X = X[mask]
y = y[mask]

# --- Fit pooled model ---
pooled_model = sm.OLS(y, X).fit()

pooled_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                     6440.
Date:                Thu, 08 Jan 2026   Prob (F-statistic):               0.00
Time:                        21:43:41   Log-Likelihood:                -5320.6
No. Observations:                 652   AIC:                         1.066e+04
Df Residuals:                     645   BIC:                         1.069e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -38.4657    173.060     -0.222      0.824    -378.295     301.364
x1           136.8611    260.295      0.526      0.599    -374.267     647.989
x2           -54.0426    173.742     -0.311      0.756    -395.211     287.126
x3          5439.2395   2760.347      1.970      0.049      18.888    1.09e+04
x4          1.063e+04   3146.436      3.377      0.001    4447.036    1.68e+04
x5         -5116.3065   2314.348     -2.211      0.027   -9660.873    -571.740
x6          2.199e+04   2396.803      9.175      0.000    1.73e+04    2.67e+04
==============================================================================
Omnibus:                      317.191   Durbin-Watson:                   1.243
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            21612.161
Skew:                          -1.302   Prob(JB):                         0.00
Kurtosis:                      31.085   Cond. No.                         178.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

###ADD LAGGED SPEND

In [ ]:
# Sort correctly for lagging
pooled_df_lag = pooled_df_clean.sort_values(
    ["dataset", "channel", "date"]
).reset_index(drop=True)

# 1-period lag of spend per dataset+channel
pooled_df_lag["spend_lag1"] = (
    pooled_df_lag
    .groupby(["dataset", "channel"])["spend"]
    .shift(1)
)

# Drop first rows where lag is missing
pooled_df_lag = pooled_df_lag.dropna(subset=["spend_lag1"]).reset_index(drop=True)

pooled_df_lag.head()


,date,channel,spend,revenue,dataset,spend_lag1
0,2022-01-09,TV,44.5,10.4,advertising,230.1
1,2022-01-16,TV,17.2,9.3,advertising,44.5
2,2022-01-23,TV,151.5,18.5,advertising,17.2
3,2022-01-30,TV,180.8,12.9,advertising,151.5
4,2022-02-06,TV,8.7,7.2,advertising,180.8


In [ ]:
# --- Force numeric types ---
pooled_df_lag["revenue"] = pd.to_numeric(pooled_df_lag["revenue"], errors="coerce")
pooled_df_lag["spend"] = pd.to_numeric(pooled_df_lag["spend"], errors="coerce")
pooled_df_lag["spend_lag1"] = pd.to_numeric(pooled_df_lag["spend_lag1"], errors="coerce")

# Drop any remaining bad rows
pooled_df_lag_clean = pooled_df_lag.dropna(
    subset=["revenue", "spend", "spend_lag1"]
).reset_index(drop=True)

# --- Build spline for current spend ---
spline_current = dmatrix(
    "bs(spend, df=5, degree=3, include_intercept=False)",
    data=pooled_df_lag_clean,
    return_type="dataframe"
)

# --- Lagged spend (linear) ---
lag_feature = pooled_df_lag_clean[["spend_lag1"]].astype(float)

# --- Dataset offset (hierarchical) ---
dataset_dummies = pd.get_dummies(
    pooled_df_lag_clean["dataset"],
    drop_first=True
).astype(float)

# --- Combine design matrix ---
X_time = pd.concat(
    [
        spline_current.reset_index(drop=True),
        lag_feature.reset_index(drop=True),
        dataset_dummies.reset_index(drop=True)
    ],
    axis=1
)

# --- Convert to numpy and clean ---
X = X_time.to_numpy(dtype=float)
y = pooled_df_lag_clean["revenue"].to_numpy(dtype=float)

mask = np.isfinite(X).all(axis=1) & np.isfinite(y)
X = X[mask]
y = y[mask]

# --- Fit pooled time-aware model ---
pooled_time_model = sm.OLS(y, X).fit()

pooled_time_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     8048.
Date:                Thu, 08 Jan 2026   Prob (F-statistic):               0.00
Time:                        21:46:10   Log-Likelihood:                -5148.3
No. Observations:                 647   AIC:                         1.031e+04
Df Residuals:                     639   BIC:                         1.035e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         22.8788    142.016      0.161      0.872    -255.996     301.754
x1           106.7059    213.776      0.499      0.618    -313.083     526.495
x2             4.9680    142.531      0.035      0.972    -274.919     284.855
x3          7749.0929   2257.905      3.432      0.001    3315.282    1.22e+04
x4          1.356e+04   2645.304      5.126      0.000    8365.641    1.88e+04
x5          8293.8783   2087.230      3.974      0.000    4195.219    1.24e+04
x6            -1.7203      0.097    -17.647      0.000      -1.912      -1.529
x7          3.165e+04   2043.297     15.491      0.000    2.76e+04    3.57e+04
==============================================================================
Omnibus:                      369.573   Durbin-Watson:                   0.891
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18764.473
Skew:                          -1.802   Prob(JB):                         0.00
Kurtosis:                      29.135   Cond. No.                     4.30e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.3e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""